<a href="https://colab.research.google.com/github/t8101349/Hugging-Face-model/blob/main/%E4%BD%BF%E7%94%A8Hugging_Face%E7%9A%84BERT_Model_%E5%81%B5%E6%B8%AC%E9%87%A3%E9%AD%9A%E7%B6%B2%E5%9D%80.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install transformers datasets scikit-learn torch pandas


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 487.4/487.4 kB 8.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 31.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 34.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 31.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 8.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 74.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 7.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [6]:
import kagglehub
import os

# Download latest version
path = kagglehub.dataset_download("ndarvind/phiusiil-phishing-url-dataset")

print("Path to dataset files:", path)

# Assuming the CSV file is in the downloaded directory
# Get a list of files in the downloaded directory
files_in_directory = os.listdir(path)

# Find the CSV file (assuming it ends with '.csv')
csv_file = [file for file in files_in_directory if file.endswith('.csv')]

# Check if a CSV file was found
if csv_file:
    # Construct the full file path
    file_path = os.path.join(path, csv_file[0])
    print("CSV file path:", file_path)
    # You can now use 'file_path' with pd.read_csv()
else:
    print("No CSV file found in the downloaded directory.")

Path to dataset files: /root/.cache/kagglehub/datasets/ndarvind/phiusiil-phishing-url-dataset/versions/1
CSV file path: /root/.cache/kagglehub/datasets/ndarvind/phiusiil-phishing-url-dataset/versions/1/PhiUSIIL_Phishing_URL_Dataset.csv


In [8]:
from google.colab import files
import pandas as pd

# 上傳 CSV 檔案
# uploaded = files.upload(path)

# 讀取 CSV 檔案（確保檔案名稱正確）
file_path = "/root/.cache/kagglehub/datasets/ndarvind/phiusiil-phishing-url-dataset/versions/1/PhiUSIIL_Phishing_URL_Dataset.csv"
df = pd.read_csv(file_path)

# 檢查數據結構
df.head()
df.info()
df['label'].value_counts()  # 查看標籤分佈


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 235795 entries, 0 to 235794
Data columns (total 55 columns):
 #   Column                      Non-Null Count   Dtype  
---  ------                      --------------   -----  
 0   URL                         235795 non-null  object 
 1   URLLength                   235795 non-null  int64  
 2   Domain                      235795 non-null  object 
 3   DomainLength                235795 non-null  int64  
 4   IsDomainIP                  235795 non-null  int64  
 5   TLD                         235795 non-null  object 
 6   URLSimilarityIndex          235795 non-null  float64
 7   CharContinuationRate        235795 non-null  float64
 8   TLDLegitimateProb           235795 non-null  float64
 9   URLCharProb                 235795 non-null  float64
 10  TLDLength                   235795 non-null  int64  
 11  NoOfSubDomain               235795 non-null  int64  
 12  HasObfuscation              235795 non-null  int64  
 13  NoOfObfuscated

,count
label,
1,134850
0,100945


In [9]:
from sklearn.model_selection import train_test_split

# 選擇 'URL' 和 'label' 欄位
df = df[['URL', 'label']].dropna()

# 70% 訓練集, 30% 測試集
train_df, test_df = train_test_split(
    df, test_size=0.3, random_state=42, stratify=df['label']
)

# 測試集再拆分為 50% 測試集 & 50% 驗證集
val_df, test_df = train_test_split(
    test_df, test_size=0.5, random_state=42, stratify=test_df['label']
)

# **🚀 資源不足: 減少訓練集、驗證集、測試集至 1/1000**
train_df = train_df.sample(frac=0.001, random_state=42)
val_df = val_df.sample(frac=0.001, random_state=42)
test_df = test_df.sample(frac=0.001, random_state=42)
# 顯示各集合大小
print(f"訓練集: {len(train_df)}, 驗證集: {len(val_df)}, 測試集: {len(test_df)}")



訓練集: 165, 驗證集: 35, 測試集: 35


In [10]:
from transformers import BertTokenizer

# 載入 BERT 預訓練 Tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

# Tokenization 方法
def tokenize_function(examples):
    return tokenizer(
        examples['URL'], padding="max_length", truncation=True, max_length=512
    )

# 對數據集進行 Tokenization
train_encodings = tokenizer(list(train_df['URL']), truncation=True, padding=True, max_length=128)
val_encodings = tokenizer(list(val_df['URL']), truncation=True, padding=True, max_length=128)
test_encodings = tokenizer(list(test_df['URL']), truncation=True, padding=True, max_length=128)



/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

In [12]:
import torch

class TextDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

# 轉換標籤
train_labels = train_df['label'].tolist()
val_labels = val_df['label'].tolist()
test_dataset=test_df['label'].tolist()

# 建立 Dataset
train_dataset = TextDataset(train_encodings, train_labels)
val_dataset = TextDataset(val_encodings, val_labels)
test_dataset = TextDataset(test_encodings, test_dataset)


In [13]:
from transformers import BertForSequenceClassification

# 載入 BERT 分類模型（適用於二元分類）
model = BertForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=2)


model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [14]:
from transformers import Trainer, TrainingArguments
from sklearn.metrics import precision_recall_fscore_support

# 啟用 Gradient Checkpointing 以降低記憶體使用
model.gradient_checkpointing_enable()
# 設定訓練參數
training_args = TrainingArguments(
    output_dir='./results',
    eval_strategy="epoch",
    per_device_train_batch_size=1,#原32因訓練演示改為1
    per_device_eval_batch_size=1,#原32因訓練演示改為1
    num_train_epochs=1,#原2因訓練演示改為1
    weight_decay=0.01,
    report_to="none",
    fp16=False,
    logging_strategy="epoch",
    logging_steps=50,
    save_total_limit=1,
    save_steps=500
)

# 計算評估指標
def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average='binary')
    return {'precision': precision, 'recall': recall, 'f1': f1}

# 建立 Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    compute_metrics=compute_metrics
)

# 禁用 W&B 日誌
import os
os.environ["WANDB_MODE"] = "disabled"

# 開始訓練
trainer.train()


Epoch,Training Loss,Validation Loss,Precision,Recall,F1
1,0.383300,0.000867,1.000000,1.000000,1.000000


TrainOutput(global_step=165, training_loss=0.38332177364464964, metrics={'train_runtime': 506.1436, 'train_samples_per_second': 0.326, 'train_steps_per_second': 0.326, 'total_flos': 10853331033600.0, 'train_loss': 0.38332177364464964, 'epoch': 1.0})

In [15]:
from torch.utils.data import Subset

# 取 1/1000 測試樣本
num_samples = max(35, 1)  # 確保至少有 1 個樣本
test_dataset = Subset(test_dataset, range(num_samples))

# 確認新的測試集大小
print(f"新的測試集大小: {len(test_dataset)}")  # 預期輸出: 35

# 在測試集上評估
test_results = trainer.evaluate(test_dataset)
print(test_results)


新的測試集大小: 35


{'eval_loss': 0.0007826137007214129, 'eval_precision': 1.0, 'eval_recall': 1.0, 'eval_f1': 1.0, 'eval_runtime': 8.0681, 'eval_samples_per_second': 4.338, 'eval_steps_per_second': 4.338, 'epoch': 1.0}


In [16]:
# 儲存模型和 tokenizer
save_directory = "./trained_model"

trainer.save_model(save_directory)
tokenizer.save_pretrained(save_directory)

# 也可以單獨儲存模型權重
import torch
torch.save(model.state_dict(), './results/pytorch_model.bin')
